*DETECCIÓN Y CONTEO DE OBJETOS EN PYTHON:*

Comando que se utiliza en entornos de Python como Jupyter Notebook, Google Colab o entornos compatibles con IPython.

*PIP INSTALL* =  gestor de paquetes de Python. Se utiliza para instalar, actualizar y gestionar paquetes (librerías) desde el repositorio oficial de Python, llamado PyPI (Python Package Index).

*ULTRALYTICS* =  paquete que contiene herramientas desarrolladas por la empresa Ultralytics, conocida especialmente por sus modelos de detección de objetos YOLO (You Only Look Once). En este caso se usa para la 'detección de objetos'.

*OPENCV-PYTHON* = instala OpenCV (Open Source Computer Vision Library) para Python, que es una de las bibliotecas más poderosas y ampliamente utilizadas en visión por computadora. En este caso se usa para procesar imágenes: detectar bordes, colores, formas, rostros, etc y trabajar con cámara en tiempo real.

In [1]:

# Instalación de librerías
!pip install ultralytics opencv-python


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Esta línea importa la clase YOLO del paquete ultralytics. Con esta clase se cargan modelos YOLO preentrenados o personalizados, para realizar tareas de Detección de objetos.

In [2]:
from ultralytics import YOLO

Importa la biblioteca OpenCV para Python, que se accede siempre como cv2. Esta biblioteca sirve para acceder a cámaras web para capturar video en tiempo real, leer imágenes o videos, mostrar imágenes en ventanas emergentes, detectar bordes, procesar píxeles etc.

In [3]:
import cv2

Carga el modelo YOLOv8 nano (yolov8n.pt), que es el más liviano y rápido. 

Este modelo ya viene preentrenado en el conjunto de datos COCO, así que puede detectar objetos comunes como personas, autos, perros, etc.

In [4]:
# Cargar modelo YOLOv8 (usa yolov8n.pt por ser más liviano)
model = YOLO("yolov8n.pt")  

Abre la cámara con índice 1. En este caso Iriun Webcam.

In [5]:
# Inicializar la cámara (0 es la cámara por defecto)
cap = cv2.VideoCapture(1)

Verifica si OpenCV logró abrir la cámara correctamente, si no fue posible, se imprime un mensaje de error y se sale del programa.

Captura una tecla para saber cuándo salir (en este caso, al presionar la tecla q).

In [7]:
# Comprobar que la cámara esté disponible
if not cap.isOpened():
    print("No se puede acceder a la cámara")
    exit()

print("Presiona 'q' para salir")

Presiona 'q' para salir


In [8]:
#Inicia un bucle infinito que captura frames desde la cámara.
#ret indica si la lectura fue exitosa.
#frame contiene la imagen capturada.
while True:
    ret, frame = cap.read()

#Si no se pudo leer un frame (por error de cámara), el bucle se rompe.
    if not ret:
        print("No se pudo leer el frame")
        break

#Realizar detección en el frame.
#verbose=False evita que se impriman logs innecesarios.
#result.boxes contiene todas las cajas detectadas.
#model.names tiene el nombre de cada clase por ID.
    results = model(frame, verbose=False)
    result = results[0]
    boxes = result.boxes
    names = model.names

# Contador de objetos
    contador = {}

#Solo se tendrán en cuenta estos objetos.
#Si se detecta algo fuera de esa lista, se ignora.
    objetos_permitidos = ["cell phone", "laptop", "tv"]

#Itera sobre todas las cajas de detección retornadas por el modelo.
#Cada box contiene información sobre un objeto detectado: clase, confianza, posición, etc.
    for box in boxes:
        cls_id = int(box.cls[0])  #Obtiene el ID numérico de la clase detectada. 
        cls_name = names[cls_id]  #Traduce el ID numérico de la clase al nombre correspondiente, usando el diccionario model.names.

        #Se usa para filtrar predicciones poco confiables.
        conf = float(box.conf[0])  #Recupera el nivel de confianza del modelo en esa predicción (valor entre 0 y 1).

        #Extrae las coordenadas de la caja que encierra el objeto detectado.
        #(x1, y1) es la esquina superior izquierda.
        #(x2, y2) es la esquina inferior derecha.
        x1, y1, x2, y2 = map(int, box.xyxy[0]) #box.xyxy[0] devuelve un arreglo de 4 números con esas coordenadas. 
        #Se convierten a int para poder usarlas con OpenCV.

        #Calcula el área de la caja de detección, para poder descartar objetos muy pequeños.
        area = (x2 - x1) * (y2 - y1) 

       # Filtrar por tipo de objeto
        if cls_name not in objetos_permitidos:
            continue 
        
        #Solo considera objetos con confianza ≥ 70%.
        if conf < 0.7:
            continue 
        
        #Ignorar objetos demasiado pequeños
        if area < 6000:
            continue 

        #Ignorar personas
        if cls_name == "person":
            continue 

        #Usa un diccionario para llevar el conteo de cada clase detectada.
        #Si el objeto ya fue contado, suma 1. Si no, lo inicia con 1.
        contador[cls_name] = contador.get(cls_name, 0) + 1


  # Dibujar caja y etiqueta 
        label = cls_name
    #Dibuja un rectángulo verde alrededor del objeto detectado.
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    #Muestra el nombre del objeto sobre la caja.
        cv2.putText(frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    #En la esquina superior izquierda, muestra el número de objetos detectados por clase.
    y_offset = 20

    #Aquí se recorre el diccionario contador, que contiene el nombre de la clase  como clave y el total de veces que fue detectada como valor.
    #Por ejemplo, si detectó 2 laptops y 1 celular, el diccionario sería: {"laptop": 2, "cell phone": 1}.
    for clase, total in contador.items():

    #Usa una f-string para combinar el nombre de la clase y el total detectado, por ejemplo: "laptop: 2".
    #Se construye el texto que se mostrará en pantalla
        texto = f"{clase}: {total}"

    #Esta línea dibuja el texto en la imagen (frame). Con sus respectivas especificaciones.
        cv2.putText(frame, texto, (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        y_offset += 20

    #Abre una ventana para mostrar los resultados en tiempo real.
    cv2.imshow("YOLOv8 Detección en Tiempo Real", frame)

    #Permite salir del bucle al presionar la tecla 'q'.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#Cierra la cámara y destruye todas las ventanas abiertas de OpenCV.
cap.release()
cv2.destroyAllWindows()